# Create JSON for HRRR Grib files

In [ ]:
so = dict(
    anon=False, requester_pays=True, default_fill_cache=False, default_cache_type='first'
)

urls = ["s3://" + p for p in ['coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00780000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00810000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00840000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00870000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00900000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00930000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00960000.nc',
             'coawst-public/rsignell/testing/gom1km/ncom_1_2021032200_00990000.nc']]


In [ ]:
%run hdf.py

In [ ]:
mzz = MultiZarrToZarr(sorted(glob.glob("*.json")),         remote_protocol="s3",
        remote_options={'requester_pays': True}, with_mf="time"
)

In [ ]:
mzz = MultiZarrToZarr("zip://*.json::gom1km.zip",         remote_protocol="s3",
        remote_options={'requester_pays': True}, with_mf="time"
)
mzz.translate("gom1km.json")

In [ ]:
mzz = MultiZarrToZarr(
    "zip://*.json::gom1km.zip",
    remote_protocol="s3",
    remote_options={'requester_pays': True},
    xarray_kwargs={
#            "preprocess": drop_coords,
        "decode_cf": False,
        "mask_and_scale": False,
        "decode_times": False,
        "decode_timedelta": False,
        "use_cftime": False,
        "decode_coords": False
    },
    with_mf='time'
)
mzz.translate("gom1km.json")

In [ ]:
%run hdf.py.orig
hdf5_multiple(urls, so)
example_ensemble('gom1km.json')

In [ ]:
%run hdf.py
example_ensemble()

In [ ]:
os.path.dirname(files[0]).split('/')[-2]

In [ ]:
os.path.dirname(files[0]).split('/')[-2]+os.path.basename(files[0])[-21:]

In [ ]:
%time example_multi(files)

In [ ]:
%%time
fslocal = fsspec.filesystem('')
json_files = fslocal.glob('hrrr.*.t*.json')

In [ ]:
json_files

In [ ]:
mzz = MultiZarrToZarr(json_files, remote_protocol="s3",
                      remote_options={"anon": True}, with_mf='time')
mzz.translate("hrrr.total.json")

Try out the resulting JSON:

In [ ]:
import xarray as xr
import hvplot.xarray

In [ ]:
%%time
so = {"anon": True}
fo = "hrrr_total.json"
fs = fsspec.filesystem("reference", fo=fo, target_options=so,
                       remote_protocol="s3", remote_options=so)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))

In [ ]:
ds.t2m.hvplot.quadmesh(x='longitude', y='latitude', 
                       rasterize=True, geo=True, cmap='turbo', tiles='OSM')